In [1]:
!pip install transformers

from google.colab import drive
drive.mount('/content/gdrive')

     |████████████████████████████████| 675kB 3.3MB/s 
     |████████████████████████████████| 890kB 17.1MB/s 
     |████████████████████████████████| 3.8MB 15.9MB/s 
     |████████████████████████████████| 1.1MB 37.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ecd3bda0cfa91f72640d0a636f82c2729e0a7611081d18f558d533fd5100ccdd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your 

In [2]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())


In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

import seaborn as sns
import string
import re    #for regex

print(tf.test.gpu_device_name())
np.set_printoptions(suppress=True)
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


/device:GPU:0
2.2.0


# Choose model

In [4]:
from transformers import BertTokenizer, BertModel


MODEL_TYPE = 'bert-base-uncased'
MAX_SIZE = 200
BATCH_SIZE = 200

tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

#### 1. Read data and tokenizer

Read tokenizer and data, as well as defining the maximum sequence length that will be used for the input to Bert (maximum is usually 512 tokens)

In [21]:
HAS_ANS = False
training_sample_count = 1000 # 4000
training_epochs = 3 # 3
test_count = 1000
running_folds = 1 # 2
MAX_SEQUENCE_LENGTH = 300

### original dataset

In [6]:
import io


df = pd.read_csv('/content/gdrive/My Drive/Licenta/dadjokes.csv', sep='<endoftext>')
# df = df.dropna()
# df['humor'] = df['humor'].astype('bool')


df_train = pd.read_csv('/content/gdrive/My Drive/Licenta/dadjokes_train.csv', sep='<endoftext>')
display(df_train.head(3))
# df_train = df_train[:training_sample_count*5]
# df_train = df_train.dropna()
# df_train['humor'] = df_train['humor'].astype('bool')

df_test = pd.read_csv('/content/gdrive/My Drive/Licenta/dadjokes_test.csv', sep='<endoftext>')
display(df_test.head(3))

df_dev = pd.read_csv('/content/gdrive/My Drive/Licenta/dadjokes_dev.csv', sep='<endoftext>')
# df_test = df_test[:test_count]
# df_test = df_test.dropna()
# df_test['humor'] = df_test['humor'].astype('bool')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,text,humor
0,I spoke to a German about walls Apparently his...,False
1,What makes a dad joke a dad joke? Isn't it app...,False
2,What do you call a rich mushroom? A fungi to b...,False


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,text,humor
0,TIFU by letting my dog give birth to a litter ...,False
1,What did 50 cent say when his grandmother was ...,False
2,After being on an episode of Say Yes To The Dr...,False


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [7]:
test_df_y = df_test.copy()
del df_test['humor']

df_sub = test_df_y.copy()

print(len(df),len(df_train),len(df_test))
display(df_train.head())
display(df_test.head())

127174 101740 12717


,text,humor
0,I spoke to a German about walls Apparently his...,False
1,What makes a dad joke a dad joke? Isn't it app...,False
2,What do you call a rich mushroom? A fungi to b...,False
3,Don't make fun of Terry &#x200B; If he kills ...,False
4,Did you hear about the kidnapping at school? I...,False


,text
0,TIFU by letting my dog give birth to a litter ...
1,What did 50 cent say when his grandmother was ...
2,After being on an episode of Say Yes To The Dr...
3,A steak pun Is rare and medium well done
4,Did you hear about the blind lady that fell in...


In [8]:
print(list(df_train.columns))

['text', 'humor']


In [9]:
output_categories = list(df_train.columns[[1]])
input_categories = list(df_train.columns[[0]])

if HAS_ANS:
    output_categories = list(df_train.columns[11:])
    input_categories = list(df_train.columns[[1,2,5]])
    

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['text']

output TARGET_COUNT:
	 1

output categories:
	 ['humor']


#### 2. Preprocessing functions

These are some functions that will be used to preprocess the raw text data into useable Bert inputs.<br>

*update 4:* credits to [Minh](https://www.kaggle.com/dathudeptrai) for this implementation. If I'm not mistaken, it could be used directly with other Huggingface transformers too! Note that due to the 2 x 512 input, it will require significantly more memory when finetuning BERT.

In [10]:
def _convert_to_transformer_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    input_ids_q, input_masks_q, input_segments_q = return_id(
        title, None, 'longest_first', max_sequence_length)
    
    input_ids_a, input_masks_a, input_segments_a = return_id(
        '', None, 'longest_first', max_sequence_length)
        
    return [input_ids_q, input_masks_q, input_segments_q,
            input_ids_a, input_masks_a, input_segments_a]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids_q, input_masks_q, input_segments_q = [], [], []
    input_ids_a, input_masks_a, input_segments_a = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.text, instance.text, instance.text

        try:
          ids_q, masks_q, segments_q, ids_a, masks_a, segments_a = \
        _convert_to_transformer_inputs(t, q, a, tokenizer, max_sequence_length)
        except Exception as e:
          print(e)
          print(t)
        
        input_ids_q.append(ids_q)
        input_masks_q.append(masks_q)
        input_segments_q.append(segments_q)
        input_ids_a.append(ids_a)
        input_masks_a.append(masks_a)
        input_segments_a.append(segments_a)
        
    return [np.asarray(input_ids_q, dtype=np.int32), 
            np.asarray(input_masks_q, dtype=np.int32), 
            np.asarray(input_segments_q, dtype=np.int32),
            np.asarray(input_ids_a, dtype=np.int32), 
            np.asarray(input_masks_a, dtype=np.int32), 
            np.asarray(input_segments_a, dtype=np.int32)]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [11]:
outputs = compute_output_arrays(df_train, output_categories)
inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
dev_inputs = compute_input_arrays(df_dev, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
dev_outputs = compute_output_arrays(df_dev, output_categories)

## 3. Create model

~~`compute_spearmanr()`~~ `mean_squared_error` is used to compute the competition metric for the validation set
<br><br>
`create_model()` contains the actual architecture that will be used to finetune BERT to our dataset.


In [12]:

def create_model():
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = BertConfig() # print(config) to see settings
    config.output_hidden_states = False # Set to True to obtain hidden states
    # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
    bert_model = TFBertModel.from_pretrained('bert-base-uncased', config=config)
    
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    q_embedding = bert_model(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
    a_embedding = bert_model(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    
    q = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
    a = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    
#     x = tf.keras.layers.Concatenate()([q, q])
    
    x = tf.keras.layers.Dropout(0.2)(q)
    
    x = tf.keras.layers.Dense(TARGET_COUNT, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, ], outputs=x)
    
    return model

#### 4. Obtain inputs and targets, as well as the indices of the train/validation splits

In [13]:
output_categories

['humor']

## 5. Training, validation and testing

Loops over the folds in gkf and trains each fold for 3 epochs --- with a learning rate of 3e-5 and batch_size of 6. A simple binary crossentropy is used as the objective-/loss-function. 

In [14]:
# Evaluation Metrics
import sklearn
def print_evaluation_metrics(y_true, y_pred, label='', is_regression=True, label2=''):
    print('==================', label2)
    ### For regression
    if is_regression:
        print('mean_absolute_error',label,':', sklearn.metrics.mean_absolute_error(y_true, y_pred))
        print('mean_squared_error',label,':', sklearn.metrics.mean_squared_error(y_true, y_pred))
        print('r2 score',label,':', sklearn.metrics.r2_score(y_true, y_pred))
        #     print('max_error',label,':', sklearn.metrics.max_error(y_true, y_pred))
        return sklearn.metrics.mean_squared_error(y_true, y_pred)
    else:
        ### FOR Classification
        print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
        print('average_precision_score',label,':', sklearn.metrics.average_precision_score(y_true, y_pred))
        print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
        print('accuracy_score',label,':', sklearn.metrics.accuracy_score(y_true, y_pred))
        print('f1_score',label,':', sklearn.metrics.f1_score(y_true, y_pred))
        
        matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
        print(matrix)
        TP,TN,FP,FN = matrix[1][1],matrix[0][0],matrix[0][1],matrix[1][0]
        Accuracy = (TP+TN)/(TP+FP+FN+TN)
        Precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1 = 2*(Recall * Precision) / (Recall + Precision)
        print('Acc', Accuracy, 'Prec', Precision, 'Rec', Recall, 'F1',F1)
        return sklearn.metrics.accuracy_score(y_true, y_pred)

print_evaluation_metrics([1,0], [0.9,0.1], '', True)
print_evaluation_metrics([1,0], [1,1], '', False)

mean_absolute_error  : 0.09999999999999999
mean_squared_error  : 0.009999999999999998
r2 score  : 0.96
balanced_accuracy_score  : 0.5
average_precision_score  : 0.5
balanced_accuracy_score  : 0.5
accuracy_score  : 0.5
f1_score  : 0.6666666666666666
[[0 1]
 [0 1]]
Acc 0.5 Prec 0.5 Rec 1.0 F1 0.6666666666666666


0.5

In [19]:
print(len(dev_inputs), len(dev_inputs[0]))

6 12717


### Loss function selection
Regression problem between 0 and 1, so binary_crossentropy and mean_absolute_error seem good.

Here are the explanations: https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/

In [25]:
min_err = 1000000
min_test = []
valid_preds = []
test_preds = []
best_model = False
# for LR in np.arange(1e-5, 2e-5, 3e-5).tolist():
for LR in [1e-5]:
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    print('LR=', LR)

    # train_inputs = [(inputs[i][:])[:training_sample_count] for i in range(len(inputs))]
    # train_outputs = (outputs[:])[:training_sample_count]
    train_inputs = inputs
    train_outputs = outputs

    valid_inputs = dev_inputs
    valid_outputs = dev_outputs

    print(np.array(train_inputs).shape, np.array(train_outputs).shape)

    K.clear_session()
    model = create_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)

    model.fit(train_inputs, train_outputs, epochs=training_epochs, batch_size=6)
    print('finished fit')
            # model.save_weights(f'bert-{fold}.h5')
    valid_preds.append(model.predict(valid_inputs, verbose=1))
    print('finished predict')
            #rho_val = compute_spearmanr_ignore_nan(valid_outputs, valid_preds[-1])
            #print('validation score = ', rho_val)
    acc = print_evaluation_metrics(np.array(valid_outputs), np.array(valid_preds[-1]), 'on #'+str(1))
    print('acc', acc)
    if acc < min_err:
        print('new acc >> ', acc)
        min_err = acc
        best_model = model
#                 min_test = model.predict(test_inputs)
#                 test_preds.append(min_test)
        print(' ')

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 1e-05
(6, 1000, 300) (1000, 1)
167/167 [==============================] - 132s 789ms/step - loss: 0.3731
finished fit
398/398 [==============================] - 478s 1s/step
finished predict
mean_absolute_error on #1 : 0.210827
mean_squared_error on #1 : 0.11071006
r2 score on #1 : 0.0014204055413742367
acc 0.11071006
new acc >>  0.11071006
 


In [26]:
print('best acc >> ', acc)

best acc >>  0.11071006


In [ ]:
len(valid_inputs[0])

In [27]:
print(valid_outputs.shape, valid_preds[-1].shape)
print_evaluation_metrics(np.array(valid_outputs), np.array(valid_preds[-1]), '')

(12717, 1) (12717, 1)
mean_absolute_error  : 0.210827
mean_squared_error  : 0.11071006
r2 score  : 0.0014204055413742367


0.11071006

In [28]:
# %%time
min_test = best_model.predict(test_inputs, verbose=1)

## use min_test
# min_test

398/398 [==============================] - 478s 1s/step


In [ ]:
df_test.to_csv('df_test.csv')
test_df_y.to_csv('test_df_y.csv')


## Regression submission

In [29]:
df_sub = test_df_y.copy()
# df_sub['pred'] = np.average(test_preds, axis=0) # for weighted average set weights=[...]
df_sub['pred'] = min_test


print_evaluation_metrics(df_sub['humor'], df_sub['pred'], '', True)


df_sub.to_csv('sub1.csv', index=False)
df_sub.head()

mean_absolute_error  : 0.21578145
mean_squared_error  : 0.11583112
r2 score  : -0.00018962442365300092


,text,humor,pred
0,TIFU by letting my dog give birth to a litter ...,False,0.116101
1,What did 50 cent say when his grandmother was ...,False,0.085329
2,After being on an episode of Say Yes To The Dr...,False,0.114239
3,A steak pun Is rare and medium well done,False,0.095976
4,Did you hear about the blind lady that fell in...,False,0.147187


## Binary submission

In [30]:
for split in np.arange(0.1, 0.99, 0.1).tolist():
    df_sub['pred_bi'] = (df_sub['pred'] > split)

    print_evaluation_metrics(df_sub['humor'], df_sub['pred_bi'], '', False, 'SPLIT on '+str(split))

    df_sub.to_csv('sub3.csv', index=False)
    df_sub.head()

================== SPLIT on 0.1
balanced_accuracy_score  : 0.5312585095761096
average_precision_score  : 0.14148288969397022
balanced_accuracy_score  : 0.5312585095761096
accuracy_score  : 0.37099944955571285
f1_score  : 0.24172907384586217
[[3443 7574]
 [ 425 1275]]
Acc 0.37099944955571285 Prec 0.1440840772968697 Rec 0.75 F1 0.24172907384586217
================== SPLIT on 0.2
balanced_accuracy_score  : 0.5005864733112997
average_precision_score  : 0.13391842051059352
balanced_accuracy_score  : 0.5005864733112997
accuracy_score  : 0.865612959031218
f1_score  : 0.0046592894583576
[[11004    13]
 [ 1696     4]]
Acc 0.865612959031218 Prec 0.23529411764705882 Rec 0.002352941176470588 F1 0.0046592894583576
================== SPLIT on 0.30000000000000004
balanced_accuracy_score  : 0.5
average_precision_score  : 0.13367932688527168
balanced_accuracy_score  : 0.5
accuracy_score  : 0.8663206731147283
f1_score  : 0.0
[[11017     0]
 [ 1700     0]]
Acc 0.8663206731147283 Prec nan Rec 0.0 F1 nan
=

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in long_scalars


================== SPLIT on 0.5
balanced_accuracy_score  : 0.5
average_precision_score  : 0.13367932688527168
balanced_accuracy_score  : 0.5
accuracy_score  : 0.8663206731147283
f1_score  : 0.0
[[11017     0]
 [ 1700     0]]
Acc 0.8663206731147283 Prec nan Rec 0.0 F1 nan
================== SPLIT on 0.6
balanced_accuracy_score  : 0.5
average_precision_score  : 0.13367932688527168
balanced_accuracy_score  : 0.5
accuracy_score  : 0.8663206731147283
f1_score  : 0.0
[[11017     0]
 [ 1700     0]]
Acc 0.8663206731147283 Prec nan Rec 0.0 F1 nan
================== SPLIT on 0.7000000000000001
balanced_accuracy_score  : 0.5
average_precision_score  : 0.13367932688527168
balanced_accuracy_score  : 0.5
accuracy_score  : 0.8663206731147283
f1_score  : 0.0
[[11017     0]
 [ 1700     0]]
Acc 0.8663206731147283 Prec nan Rec 0.0 F1 nan
================== SPLIT on 0.8
balanced_accuracy_score  : 0.5
average_precision_score  : 0.13367932688527168
balanced_accuracy_score  : 0.5
accuracy_score  : 0.86632067

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
best_model.save('/content/gdrive/My Drive/dadjokes_model')
# loaded_model = tf.keras.models.load_model('/tmp/model')

INFO:tensorflow:Assets written to: dadjokes_model/assets
  adding: content/dadjokes_model/ (stored 0%)
  adding: content/dadjokes_model/variables/ (stored 0%)
  adding: content/dadjokes_model/variables/variables.index (deflated 81%)
  adding: content/dadjokes_model/variables/variables.data-00000-of-00002 (deflated 36%)
  adding: content/dadjokes_model/variables/variables.data-00001-of-00002 (deflated 19%)
  adding: content/dadjokes_model/assets/ (stored 0%)
  adding: content/dadjokes_model/saved_model.pb (deflated 92%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>